In [2]:
import torch#源于master版本
from torch.ao.quantization import (
  get_default_qconfig_mapping,
  get_default_qat_qconfig_mapping,
  QConfigMapping,
)
import torch.quantization.quantize_fx as quantize_fx
import copy
import torchvision.datasets as datasets

import sys
sys.path.append("../") 
sys.path.append("../utils/")
# sys.path.append('MyTransformer/utils')
from Mymodels import Vit
from utils import build_transform

from utils import validate


In [3]:
In_Channels=3
Embed_Dim=384
Picture_Size=224
Patch_Size=16
Num_Class=3
Num_Heads=6
Encoder_Layers=6
Onnx_Export_Path='Exported_Fx_Onnx.onnx'
model_fp=Vit(In_Channels=In_Channels,Out_Channels=Embed_Dim,Picture_Size=Picture_Size,Patch_Size=Patch_Size
,Num_Class=Num_Class,Num_Heads=Num_Heads,Encoder_Layers=Encoder_Layers)

In [4]:
def calibrate(val_loader,model, device):

    # lr = 0.0001
    # optimizer = optim.Adam(model.parameters(), lr=lr)
    # val_start_time = end = time.time()
    # running_loss=0
    model.eval()
    with torch.no_grad():
        for i, (data, target) in enumerate(val_loader):#一次验证一个batch，每个target就有batch的维度
            data = data.to(device)
            print("calibrate times:",i)
            # with torch.no_grad():
            output = model(data)


In [5]:
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)
crop_pct = 0.9
train_transform = build_transform(mean=mean, std=std, crop_pct=crop_pct)
val_dataset = datasets.ImageFolder('E:/Transformer/DataSets/imagenet/Mini_Train/val',train_transform)
train_dataset = datasets.ImageFolder('E:/Transformer/DataSets/imagenet/Mini_Train/train',train_transform)
model_save_path='Saved.pth'
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=1,#args.val_batchsize,
    shuffle=False,
    num_workers=8,#args.num_workers,
    pin_memory=True,
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,#args.val_batchsize,
    shuffle=True,
    num_workers=8,#args.num_workers,
    pin_memory=True,
)

c:\Users\25073\.conda\envs\YoloV5\lib\site-packages\torchvision\transforms\transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [6]:
model_to_quantize = copy.deepcopy(model_fp)
qconfig_mapping = get_default_qconfig_mapping("qnnpack")
model_to_quantize.eval()
# prepare
example_inputs=torch.rand(1,3,224,224)
model_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_mapping, example_inputs)
# calibrate
model_prepared.to('cuda')
calibrate(val_loader,model_prepared,'cuda')
# quantize

model_quantized = quantize_fx.convert_fx(model_prepared)


calibrate times: 0
calibrate times: 1
calibrate times: 2
calibrate times: 3
calibrate times: 4
calibrate times: 5
calibrate times: 6
calibrate times: 7
calibrate times: 8
calibrate times: 9
calibrate times: 10
calibrate times: 11
calibrate times: 12
calibrate times: 13
calibrate times: 14
calibrate times: 15
calibrate times: 16
calibrate times: 17
calibrate times: 18
calibrate times: 19
calibrate times: 20
calibrate times: 21
calibrate times: 22
calibrate times: 23
calibrate times: 24
calibrate times: 25
calibrate times: 26
calibrate times: 27
calibrate times: 28
calibrate times: 29
calibrate times: 30
calibrate times: 31
calibrate times: 32
calibrate times: 33
calibrate times: 34
calibrate times: 35
calibrate times: 36
calibrate times: 37
calibrate times: 38
calibrate times: 39
calibrate times: 40
calibrate times: 41
calibrate times: 42
calibrate times: 43
calibrate times: 44
calibrate times: 45
calibrate times: 46
calibrate times: 47
calibrate times: 48
calibrate times: 49
calibrate 

In [8]:
#validate:
import torch.nn as nn
model_quantized.to('cpu')
criterion = nn.CrossEntropyLoss().to('cpu')
validate(1,val_loader,model_quantized,criterion,'cpu')


RuntimeError: getCudnnDataTypeFromScalarType() not supported for QUInt8

In [32]:

# model_quantized.print_readable()
print(model_quantized)

GraphModule(
  (patch_embed): Module(
    (proj): QuantizedConv2d(3, 384, kernel_size=(16, 16), stride=(16, 16), scale=0.015282480977475643, zero_point=128)
  )
  (Encoders): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
        )
        (linear1): Linear(in_features=384, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=384, bias=True)
        (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=384,

In [1]:
#val
a=10.232
print(str(a)[1])



.
